## Deep Learning Prototype

## Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Load Training CSV File

Displays dataframe for verification purposes.

In [2]:
# Load the training CSV file
csv_file = 'image_labels/airspeed_labels.csv'
training_df = pd.read_csv(csv_file)

image_directory = 'media/airspeed_frames'
training_df['image_path'] = training_df['image_path'].apply(lambda x: os.path.join(image_directory, x))

training_df


,image_path,airspeed
0,airspeed_frames/dive_fixed_A10C_0000.jpg,190
1,airspeed_frames/dive_fixed_A10C_0001.jpg,190
2,airspeed_frames/dive_fixed_A10C_0002.jpg,190
3,airspeed_frames/dive_fixed_A10C_0003.jpg,190
4,airspeed_frames/dive_fixed_A10C_0004.jpg,190
...,...,...
3482,airspeed_frames/takeoff_fixed_A10C_1591.jpg,190
3483,airspeed_frames/takeoff_fixed_A10C_1592.jpg,190
3484,airspeed_frames/takeoff_fixed_A10C_1593.jpg,190
3485,airspeed_frames/takeoff_fixed_A10C_1594.jpg,190


## Helper Function - Load Image

In [3]:
def load_image(file_path):
    image = cv2.imread(file_path)

    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

## Load Images and apply labels from CSV

In [5]:
# Load images into a NumPy array
images = np.array([load_image(file_path) for file_path in training_df['image_path']])


# Encode the labels (assuming 'vertical_speed' is the column containing labels)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(training_df['airspeed'])

# Optionally, one-hot encode the labels (if using categorical cross-entropy)
num_classes = len(label_encoder.classes_)
one_hot_labels = tf.keras.utils.to_categorical(encoded_labels, num_classes)


# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)

## Epoch Notes

Best for airspeed - Epoch 13

## Execute - Run Training Model

In [7]:
# Define the model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(110, 120, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Adjust num_classes as needed
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use appropriate loss function
              metrics=['accuracy'])

# Load and preprocess your dataset
# You can use TensorFlow's `ImageDataGenerator` for data augmentation and preprocessing.

# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

# Save the trained model
model.save('airspeed_model.h5')


Epoch 1/20
88/88 [==============================] - 6s 63ms/step - loss: 1.7384 - accuracy: 0.5324 - val_loss: 0.5411 - val_accuracy: 0.8138
Epoch 2/20
88/88 [==============================] - 6s 64ms/step - loss: 0.3866 - accuracy: 0.8670 - val_loss: 0.3470 - val_accuracy: 0.8711
Epoch 3/20
88/88 [==============================] - 6s 64ms/step - loss: 0.2469 - accuracy: 0.9114 - val_loss: 0.2115 - val_accuracy: 0.9298
Epoch 4/20
88/88 [==============================] - 6s 65ms/step - loss: 0.1958 - accuracy: 0.9319 - val_loss: 0.1733 - val_accuracy: 0.9484
Epoch 5/20
88/88 [==============================] - 6s 65ms/step - loss: 0.1655 - accuracy: 0.9347 - val_loss: 0.1900 - val_accuracy: 0.9384
Epoch 6/20
88/88 [==============================] - 6s 65ms/step - loss: 0.1699 - accuracy: 0.9369 - val_loss: 0.1448 - val_accuracy: 0.9456
Epoch 7/20
88/88 [==============================] - 6s 66ms/step - loss: 0.1194 - accuracy: 0.9573 - val_loss: 0.1231 - val_accuracy: 0.9484
Epoch 8/20
88

/opt/homebrew/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
